# Redes Neuronales usando Tensorflow

### 1. Importación de librerias

In [ ]:
#!pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

### 2. Pre-Procesamiento de Datos

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [ ]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
print("Cantidad de Datos en la Base: ",len(dataset))

Cantidad de Datos en la Base:  10000


##### Extraer las Variables independientes y dependientes

In [ ]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


##### Transformación de Variables Categoricas

In [ ]:
# Transformación de la columna "Gender"
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [ ]:
# Transformación de la columna "Geography"
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


##### Split de datos de Entrenamiento y testeo

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

##### Estandarización

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### 3. Construcción de la Red Neuronal

In [ ]:
# Inicialización de la red neuronal
rna = tf.keras.models.Sequential()

In [ ]:
# Añadir la capa de entrada y la primera capa oculta
rna.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [ ]:
# Añadir una segunda capa oculta
rna.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [ ]:
# Añadir la capa de salida
rna.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### 4. Entrenamiento de la Red Neuronal

https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/

In [ ]:
# Compilación de la red Neuronal
rna.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Entrenamiento de la Red Neuronal
rna.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6461 - accuracy: 0.6554
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5311 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5063 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4959 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4883 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4807 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4724 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4634 - accuracy: 0.7968
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4544 - accuracy: 0.7989
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4459 - accura

In [ ]:
# Predicción de resultados
y_pred = rna.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [ ]:
# Construcción de la matriz de confusión
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1507   88]
 [ 205  200]]


0.8535

In [ ]:
# especificidad = 0.95
# sensibilidad = 0.52 // balancear la data